In [1]:
from huggingface_hub import login
import os
login(os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
import datasets

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

In [3]:
from tqdm import tqdm
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]}) for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size=200,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

# Split docs and keep only unique ones
print("Splitting documents...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[new_doc.page_content] = True
            docs_processed.append(new_doc)

print("Embedding documents... This should take a few minutes (5 minutes on MacBook with M1 Pro)")
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")
vectordb = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE,
)

Splitting documents...


100%|██████████| 2647/2647 [01:18<00:00, 33.60it/s]
/tmp/ipykernel_29866/3239472072.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")


Embedding documents... This should take a few minutes (5 minutes on MacBook with M1 Pro)


In [4]:
from smolagents import Tool
from langchain_core.vectorstores import VectorStore


class RetrieverTool(Tool):
    name = "retriever"
    description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, vectordb: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.vectordb.similarity_search(
            query,
            k=7,
        )

        return "\nRetrieved documents:\n" + "".join(
            [f"===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
        )

In [8]:
from smolagents import HfApiModel, ToolCallingAgent

model = HfApiModel()
# model = HfApiModel()

retriever_tool = RetrieverTool(vectordb)
agent = ToolCallingAgent(tools=[retriever_tool], model=model)

In [9]:
agent_output = agent.run("How can I push a model to the Hub?")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How can I push a model to the Hub?                                                                              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'retriever' with arguments: {'query': 'How to push a model to the Hub'}                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Retrieved documents:
===== Document 0 =====
# Step 7. Push everything to the Hub
    api.upload_folder(
        repo_id=repo_id,
        folder_path=repo_local_path,
        path_in_repo=".",
    )

    print("Your model is pushed to the Hub. You can view your model here: ", repo_url)
```

### .

By using `push_to_hub` **you evaluate, record a replay, generate a model card of your agent and push it to the 
Hub**.===== Document 1 =====
Let's pretend we've now fine-tuned the model. The next step would be to push it to the Hub! We can do this with the
`timm.models.hub.push_to_hf_hub` function.

```py
>>> model_cfg = dict(labels=|'a', 'b', 'c', 'd'])
>>> timm.models.hub.push_to_hf_hub(model, 'resnet18-random', model_config=model_cfg)
```

Running the above would push the model to `<your-username>/resnet18-random` on the Hub. You can now share this 
model with your friends, or use it in your own code!

## Loading a Model===== Document 2 =====
```py
>>> trainer.push_to_hub()
```
</pt>
<tf>
Share a model to the Hub with |`PushToHubCallback`]. In the |`PushToHubCallback`] function, add:

- An output directory for your model.
- A tokenizer.
- The `hub_model_id`, which is your Hub username and model name.

```py
>>> from transformers import PushToHubCallback

>>> push_to_hub_callback = PushToHubCallback(
...     output_dir="./your_model_save_path", tokenizer=tokenizer, hub_model_id="your-username/my-awesome-model"
... )
```===== Document 3 =====
. The second way to upload a model, though, is to call model.push_to_hub(). So this is more of a once-off method - 
it's not called regularly during training. You can just call this manually whenever you want to upload a model to 
the hub. So we recommend running this after the end of training, just to make sure that you have a commit message 
just to guarantee that this was the final version of the model at the end of training. And it just makes sure that 
you're working with the definitive end-of-training model and not accidentally using a model that's from a 
checkpoint somewhere along the way===== Document 4 =====
--push_to_hub
```===== Document 5 =====
processor.push_to_hub(hub_model_id)
trainer.push_to_hub(**kwargs)
```

# 4. Inference

Now comes the exciting part, using our fine-tuned model! In this section, we'll show how you can load your model 
from the hub and use it for inference.===== Document 6 =====
Finally, if you want, you can push your model up to the hub. Here, we'll push it up if you specified 
`push_to_hub=True` in the training configuration. Note that in order to push to hub, you'll have to have git-lfs 
installed and be logged into your Hugging Face account (which can be done via `huggingface-cli login`).

```python
kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "image-classification",
    "dataset": 'beans',
    "tags": |'image-classification'],
}

[Step 0: Duration 0.15 seconds| Input tokens: 1,211 | Output tokens: 29]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'To push a model to the Hugging Face Hub, you can use   │
│ several methods depending on the library and setup you are using. Here are a few common approaches:\\n1. Using  │
│ the Hugging Face `transformers` library:\\n   ```python\\n   trainer.push_to_hub()\\n   ```\\n   or manually    │
│ pushing the model:\\n   ```python\\n   model.push_to_hub()\\x0c   ```\\n2. Using `timm` library:\\x0c           │
│ ```python\\n   timm.models.hub.push_to_hf_hub(model, \'resnet18-random\', model_config=model_cfg)\\">\\'}       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: To push a model to the Hugging Face Hub, you can use several methods depending on the library and 
setup you are using. Here are a few common approaches:\n1. Using the Hugging Face `transformers` library:\n   
```python\n   trainer.push_to_hub()\n   ```\n   or manually pushing the model:\n   ```python\n   
model.push_to_hub()\x0c   ```\n2. Using `timm` library:\x0c   ```python\n   timm.models.hub.push_to_hf_hub(model, 
'resnet18-random', model_config=model_cfg)\">\

[Step 1: Duration 0.10 seconds| Input tokens: 3,195 | Output tokens: 184]

Final output:
To push a model to the Hugging Face Hub, you can use several methods depending on the library and setup you are using. Here are a few common approaches:\n1. Using the Hugging Face `transformers` library:\n   ```python\n   trainer.push_to_hub()\n   ```\n   or manually pushing the model:\n   ```python\n   model.push_to_hub()\x0c   ```\n2. Using `timm` library:\x0c   ```python\n   timm.models.hub.push_to_hf_hub(model, 'resnet18-random', model_config=model_cfg)\">\
